In [5]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import os
import torch.nn as nn
import torchvision.models as models

In [6]:
import cv2
import frame_constructor as fc

In [7]:
#from google.colab import drive
#drive.mount('/content/drive')


In [8]:
import torch.nn as nn
import torchvision.models as models

def build_feature_extractor():
    mobilenet = models.mobilenet_v2(weights="IMAGENET1K_V1")
    mobilenet.eval()
    # Remove classifier head, add adaptive pool and flatten
    feature_extractor = nn.Sequential(
        *list(mobilenet.children())[:-1],      
        nn.AdaptiveAvgPool2d((1, 1)),         
        nn.Flatten()
    )
    return feature_extractor

# Usage:
feature_extractor = build_feature_extractor()

In [9]:
'''
import os

# correct path
dataset_path = "/content/drive/MyDrive/dog_dataset_split/train"

# list label types (aggressive / notaggressive)
label_types = os.listdir(dataset_path)
print(label_types)
'''

'\nimport os\n\n# correct path\ndataset_path = "/content/drive/MyDrive/dog_dataset_split/train"\n\n# list label types (aggressive / notaggressive)\nlabel_types = os.listdir(dataset_path)\nprint(label_types)\n'

In [10]:

'''import os

train_dir = '/content/drive/MyDrive/dog_dataset_split/train'
class_folders = os.listdir(train_dir)
print(class_folders)
'''

"import os\n\ntrain_dir = '/content/drive/MyDrive/dog_dataset_split/train'\nclass_folders = os.listdir(train_dir)\nprint(class_folders)\n"

In [11]:
'''rooms = []
for item in class_folders:
    class_folder = os.path.join(train_dir, item)
    if os.path.isdir(class_folder):  # Only proceed if it's a folder
        all_files = os.listdir(class_folder)
        for fname in all_files:
            file_path = os.path.join(class_folder, fname)
            rooms.append((item, file_path))

# Build a dataframe
import pandas as pd
train_df = pd.DataFrame(rooms, columns=['tag', 'video_path'])
print(train_df.head())
print(train_df.tail())

df = train_df.loc[:,['video_path','tag']]
df
df.to_csv('train.csv')'''

"rooms = []\nfor item in class_folders:\n    class_folder = os.path.join(train_dir, item)\n    if os.path.isdir(class_folder):  # Only proceed if it's a folder\n        all_files = os.listdir(class_folder)\n        for fname in all_files:\n            file_path = os.path.join(class_folder, fname)\n            rooms.append((item, file_path))\n\n# Build a dataframe\nimport pandas as pd\ntrain_df = pd.DataFrame(rooms, columns=['tag', 'video_path'])\nprint(train_df.head())\nprint(train_df.tail())\n\ndf = train_df.loc[:,['video_path','tag']]\ndf\ndf.to_csv('train.csv')"

In [12]:
'''import os
import pandas as pd

test_dir = '/content/drive/MyDrive/dog_dataset_split/test'
activity_types = os.listdir(test_dir)
print("Types of activities found:", activity_types)

rooms = []

for activity in activity_types:
    activity_folder = os.path.join(test_dir, activity)
    if os.path.isdir(activity_folder):  # Only if it's a folder
        for fname in os.listdir(activity_folder):
            # Absolute file path on Drive
            full_path = os.path.join(activity_folder, fname)
            # OR if you want a relative path: os.path.join('dog_dataset_split/test', activity, fname)
            rooms.append((activity, full_path))

# Build a dataframe
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_path'])
print(test_df.head())
print(test_df.tail())

# If you specifically need only two columns:
df = test_df.loc[:, ['video_path', 'tag']]
df.to_csv('test.csv', index=False)
print("CSV saved as test.csv")'''


'import os\nimport pandas as pd\n\ntest_dir = \'/content/drive/MyDrive/dog_dataset_split/test\'\nactivity_types = os.listdir(test_dir)\nprint("Types of activities found:", activity_types)\n\nrooms = []\n\nfor activity in activity_types:\n    activity_folder = os.path.join(test_dir, activity)\n    if os.path.isdir(activity_folder):  # Only if it\'s a folder\n        for fname in os.listdir(activity_folder):\n            # Absolute file path on Drive\n            full_path = os.path.join(activity_folder, fname)\n            # OR if you want a relative path: os.path.join(\'dog_dataset_split/test\', activity, fname)\n            rooms.append((activity, full_path))\n\n# Build a dataframe\ntest_df = pd.DataFrame(data=rooms, columns=[\'tag\', \'video_path\'])\nprint(test_df.head())\nprint(test_df.tail())\n\n# If you specifically need only two columns:\ndf = test_df.loc[:, [\'video_path\', \'tag\']]\ndf.to_csv(\'test.csv\', index=False)\nprint("CSV saved as test.csv")'

In [13]:
train_df = pd.read_csv("train_aug.csv")
train_df = train_df.sample(frac= 0.6, random_state=42).reset_index(drop=True)

#train_df= train_df[:300]
test_df = pd.read_csv("test_aug.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.head(10)

Total videos for training: 562
Total videos for testing: 144


,path,label
0,C:\Users\aliza\Desktop\og_behaviour\dog_datase...,not_aggressive
1,C:\Users\aliza\Desktop\og_behaviour\dog_datase...,not_aggressive
2,C:\Users\aliza\Desktop\og_behaviour\dog_datase...,aggressive
3,C:\Users\aliza\Desktop\og_behaviour\dog_datase...,not_aggressive
4,C:\Users\aliza\Desktop\og_behaviour\dog_datase...,not_aggressive
5,C:\Users\aliza\Desktop\og_behaviour\dog_datase...,aggressive
6,C:\Users\aliza\Desktop\og_behaviour\dog_datase...,aggressive
7,C:\Users\aliza\Desktop\og_behaviour\dog_datase...,aggressive
8,C:\Users\aliza\Desktop\og_behaviour\dog_datase...,aggressive
9,C:\Users\aliza\Desktop\og_behaviour\dog_datase...,aggressive


In [14]:
# Count missing labels
print("Missing labels:", train_df['label'].isnull().sum())


Missing labels: 0


In [15]:
train_df.isnull().sum()

path     0
label    0
dtype: int64

In [16]:
#train label encoding
import pandas as pd
import numpy as np
import torch
train_df=pd.read_csv('train_aug.csv')
 #convert classlabels to label encoding
train_df = train_df.dropna(subset=['label']).reset_index(drop=True)
train_df['label'] = train_df['label'].map({'aggressive': 1, 'not_aggressive': 0})
label= train_df['label'].astype(int)
label=torch.tensor(label)
#print(label,type(label))
train_df.head()


#for test


test_df=pd.read_csv('test_aug.csv')
#convert class labels to label encoding
test_df = test_df.dropna(subset=['label']).reset_index(drop=True)
test_df['label'] = test_df['label'].map({'aggressive': 1, 'not_aggressive': 0})
label= test_df['label'].astype(int)
label=torch.tensor(label)
#print(label,type(label))
test_df.head()

check=train_df['path'][0]

In [17]:
import os

video_path = check
if os.path.isfile(video_path):
    print(f"{video_path} exists.")
else:
    print(f"{video_path} does NOT exist!")


C:\Users\aliza\Desktop\og_behaviour\dog_dataset_augmented\train\not_aggressive\chewing_10_preview_4_flip.mp4 exists.


In [18]:
import cv2

cap = cv2.VideoCapture(video_path)
is_opened = cap.isOpened()
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()

if is_opened and frame_count > 0:
    print(f"{video_path} is a valid video with {frame_count} frames.")
else:
    print(f"{video_path} is NOT a valid video or has no frames!")


C:\Users\aliza\Desktop\og_behaviour\dog_dataset_augmented\train\not_aggressive\chewing_10_preview_4_flip.mp4 is a valid video with 634 frames.


In [19]:
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 10
MAX_SEQ_LENGTH = 300 # Maximum number of frames to use per video
NUM_FEATURES = 1280 # Number of features extracted by MobileNetV2

In [20]:
import torch
import torch.nn.functional as F
import numpy as np

def remove_similar_frames(frames, feature_extractor, threshold=0.95, verbose=True):
    """
    Removes frames that are too similar based on cosine similarity of extracted features.

    Args:
        frames (np.ndarray): Shape (num_frames, H, W, C).
        feature_extractor (nn.Module): Pretrained feature extractor.
        threshold (float): Cosine similarity threshold (>= means drop).
        verbose (bool): Print how many frames were kept.

    Returns:
        np.ndarray: Filtered frames.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"

    num_frames = len(frames)
    if num_frames == 0:
        if verbose:
            print("[Frame filter] No frames to process.")
        return frames

    kept_frames = [frames[0]]
    prev_feat = None

    for i, frame in enumerate(frames):
        # Convert to tensor and normalize
        frame_tensor = torch.from_numpy(frame).permute(2, 0, 1).unsqueeze(0).float().to(device) / 255.0
        mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(1, 3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225], device=device).view(1, 3, 1, 1)
        frame_tensor = (frame_tensor - mean) / std

        with torch.no_grad():
            feat = feature_extractor(frame_tensor)   # (1, D)
            feat = F.normalize(feat, dim=-1)         # cosine similarity uses normalized vectors

        if prev_feat is not None:
            sim = torch.sum(prev_feat * feat).item()
            if sim < threshold:  # keep only if different enough
                kept_frames.append(frame)
                prev_feat = feat
        else:
            prev_feat = feat

    kept_frames = np.array(kept_frames)

    # Print debug info
    if verbose:
        print(f"[Frame filter] Kept {len(kept_frames)} / {num_frames} frames "
              f"({100 * len(kept_frames) / num_frames:.1f}%) after threshold={threshold}")

    return kept_frames


In [21]:
import frame_constructor as fc


def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["path"].values.tolist()
    labels = torch.tensor(df["label"].values, dtype=torch.long)

    frame_masks = torch.zeros((num_samples, MAX_SEQ_LENGTH), dtype=torch.bool)
    frame_features = torch.zeros((num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype=torch.float32)
    
    for idx, path in enumerate(video_paths):
        video_path = path if os.path.isabs(path) else os.path.join(root_dir, path)

        frames = fc.frame_cons(video_path)
        frames = remove_similar_frames(frames, feature_extractor)

        if frames.size == 0:
            print(f"Warning: No frames extracted from {video_path}")
            continue

        video_length = frames.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)

        temp_features = torch.zeros((MAX_SEQ_LENGTH, NUM_FEATURES), dtype=torch.float32)
        
        for i in range(length):
            frame = frames[i]
            frame_tensor = torch.from_numpy(frame).permute(2, 0, 1).unsqueeze(0).float() / 255.0
            mean = torch.tensor([0.485, 0.456, 0.406], device=frame_tensor.device).view(1, 3, 1, 1)
            std = torch.tensor([0.229, 0.224, 0.225], device=frame_tensor.device).view(1, 3, 1, 1)
            frame_tensor = (frame_tensor - mean) / std

            with torch.no_grad():
                feat = feature_extractor(frame_tensor)
            temp_features[i] = feat.squeeze(0)

        frame_features[idx] = temp_features
        frame_masks[idx, :length] = True
        print(f'done for {idx}')

    return (frame_features, frame_masks), labels


In [22]:
'''
import frame_constructor as fc
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["path"].values.tolist()
    labels = torch.tensor(df["label"].values, dtype=torch.long)

    # Preallocate tensors for the whole dataset
    frame_masks = torch.zeros((num_samples, MAX_SEQ_LENGTH), dtype=torch.bool)
    frame_features = torch.zeros((num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype=torch.float32)
    
    for idx, path in enumerate(video_paths):
        # Use absolute path if present, else combine with root_dir
        if os.path.isabs(path):
            video_path = path
        else:
            video_path = os.path.join(root_dir, path)

        frames = fc.frame_cons(video_path)
        if frames.size == 0:
            print(f"Warning: No frames extracted from {video_path}")  # shape: (video_length, H, W, C) as numpy array
        video_length = frames.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        
        temp_features = torch.zeros((MAX_SEQ_LENGTH, NUM_FEATURES), dtype=torch.float32)
        
        for i in range(length):
            frame = frames[i]  # (H, W, C), numpy
            frame_tensor = torch.from_numpy(frame).permute(2, 0, 1).unsqueeze(0).float() / 255.0
            mean = torch.tensor([0.485, 0.456, 0.406], device=frame_tensor.device).view(1, 3, 1, 1)
            std = torch.tensor([0.229, 0.224, 0.225], device=frame_tensor.device).view(1, 3, 1, 1)
            frame_tensor = (frame_tensor - mean) / std

            with torch.no_grad():
                feat = feature_extractor(frame_tensor)  # output: (1, 2048)
            temp_features[i] = feat.squeeze(0)

        # Assign features and mask
        frame_features[idx] = temp_features
        frame_masks[idx, :length] = True
        print(f'done for {idx}')

    return (frame_features, frame_masks), labels
'''


'\nimport frame_constructor as fc\ndef prepare_all_videos(df, root_dir):\n    num_samples = len(df)\n    video_paths = df["path"].values.tolist()\n    labels = torch.tensor(df["label"].values, dtype=torch.long)\n\n    # Preallocate tensors for the whole dataset\n    frame_masks = torch.zeros((num_samples, MAX_SEQ_LENGTH), dtype=torch.bool)\n    frame_features = torch.zeros((num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype=torch.float32)\n    \n    for idx, path in enumerate(video_paths):\n        # Use absolute path if present, else combine with root_dir\n        if os.path.isabs(path):\n            video_path = path\n        else:\n            video_path = os.path.join(root_dir, path)\n\n        frames = fc.frame_cons(video_path)\n        if frames.size == 0:\n            print(f"Warning: No frames extracted from {video_path}")  # shape: (video_length, H, W, C) as numpy array\n        video_length = frames.shape[0]\n        length = min(MAX_SEQ_LENGTH, video_length)\n        \n      

In [23]:
(train_features, train_masks), train_labels = prepare_all_videos(train_df, " ")
(test_features, test_masks), test_labels = prepare_all_videos(test_df, " ")
#156
#167

[Frame filter] Kept 36 / 634 frames (5.7%) after threshold=0.95
done for 0
[Frame filter] Kept 55 / 300 frames (18.3%) after threshold=0.95
done for 1
[Frame filter] Kept 29 / 332 frames (8.7%) after threshold=0.95
done for 2
[Frame filter] Kept 109 / 1001 frames (10.9%) after threshold=0.95
done for 3
[Frame filter] Kept 540 / 3450 frames (15.7%) after threshold=0.95
done for 4
[Frame filter] Kept 56 / 360 frames (15.6%) after threshold=0.95
done for 5
[Frame filter] Kept 92 / 414 frames (22.2%) after threshold=0.95
done for 6
[Frame filter] Kept 45 / 429 frames (10.5%) after threshold=0.95
done for 7
[Frame filter] Kept 1084 / 2682 frames (40.4%) after threshold=0.95
done for 8
[Frame filter] Kept 21 / 505 frames (4.2%) after threshold=0.95
done for 9
[Frame filter] Kept 1192 / 5138 frames (23.2%) after threshold=0.95
done for 10
[Frame filter] Kept 83 / 316 frames (26.3%) after threshold=0.95


KeyboardInterrupt: 

In [ ]:
print((train_masks.sum(dim=1) == 0).sum().item(), "samples have zero valid frames in training data")
print((test_masks.sum(dim=1) == 0).sum().item(), "samples have zero valid frames in testing data")

0 samples have zero valid frames in training data
0 samples have zero valid frames in testing data


In [ ]:
#lstm sequence model
import torch
import torch.nn as nn
import torch.nn.functional as F

class LSTMSequenceModel(nn.Module):
    def __init__(self, num_features, max_seq_length, num_classes):
        super().__init__()
        self.lstm1 = nn.LSTM(input_size=num_features, hidden_size=16, num_layers=1, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=16, hidden_size=8, num_layers=1, batch_first=True)
        self.dropout = nn.Dropout(0.4)
        self.fc1 = nn.Linear(8, 8)
        self.fc2 = nn.Linear(8, num_classes)
    def forward(self, x, mask=None):
        # x: (batch_size, seq_len, num_features)
        lengths = mask.sum(dim=1) if mask is not None else torch.full((x.size(0),), x.size(1), dtype=torch.long).to(x.device)
        lengths = torch.clamp(lengths, min=1)  # Prevent zero lengths
        # Pack padded sequence for the LSTM
        packed = nn.utils.rnn.pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, (h1, _) = self.lstm1(packed)
        packed_output, (h2, _) = self.lstm2(packed_output)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True, total_length=x.size(1))

        # Get the last valid output in each sequence according to actual length
        idx = (lengths - 1).unsqueeze(1).unsqueeze(2).expand(-1, 1, output.shape[2])
        last_outputs = output.gather(1, idx).squeeze(1)
        x = self.dropout(last_outputs)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        #print(x.shape) 16,2
        return x  # logits



In [ ]:
from torch.utils.data import TensorDataset, DataLoader


train_dataset = TensorDataset(train_features, train_masks, train_labels)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = TensorDataset(test_features, test_masks, test_labels)  # if using validation set
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
model = LSTMSequenceModel(num_features=NUM_FEATURES, max_seq_length=MAX_SEQ_LENGTH, num_classes=num_classes)

NameError: name 'NUM_CLASSES' is not defined

In [ ]:
NUM_FEATURES = train_features.shape[2]
MAX_SEQ_LENGTH = train_features.shape[1]
NUM_CLASSES = len(torch.unique(train_labels))
model = LSTMSequenceModel(num_features=NUM_FEATURES, max_seq_length=MAX_SEQ_LENGTH, num_classes=NUM_CLASSES)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)


NameError: name 'train_features' is not defined

In [ ]:
import torch.nn as nn
EPOCHS = 20
LEARNING_RATE = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

best_val_acc = 0.0
best_model_state = None

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for batch in train_loader:
        features, masks, labels = [x.to(device) for x in batch]  # Unpack batch
        optimizer.zero_grad()
        logits = model(features, masks)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * features.size(0)
        preds = logits.argmax(1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

    train_acc = train_correct / train_total
    avg_train_loss = train_loss / train_total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for batch in val_loader:
            features, masks, labels = [x.to(device) for x in batch]
            logits = model(features, masks)
            preds = logits.argmax(1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    val_acc = val_correct / val_total

    print(f"Epoch {epoch+1}/{EPOCHS} - Train loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict().copy()

# Restore best model
if best_model_state:
    model.load_state_dict(best_model_state)


Epoch 1/20 - Train loss: 0.4685, Train Acc: 0.8217, Val Acc: 0.7630
Epoch 2/20 - Train loss: 0.4601, Train Acc: 0.8290, Val Acc: 0.7861
Epoch 3/20 - Train loss: 0.3997, Train Acc: 0.8652, Val Acc: 0.7977
Epoch 4/20 - Train loss: 0.5198, Train Acc: 0.7913, Val Acc: 0.7803
Epoch 5/20 - Train loss: 0.4608, Train Acc: 0.8319, Val Acc: 0.8382
Epoch 6/20 - Train loss: 0.4166, Train Acc: 0.8580, Val Acc: 0.6474
Epoch 7/20 - Train loss: 0.4339, Train Acc: 0.8449, Val Acc: 0.8497
Epoch 8/20 - Train loss: 0.4132, Train Acc: 0.8565, Val Acc: 0.8150
Epoch 9/20 - Train loss: 0.3870, Train Acc: 0.8667, Val Acc: 0.8208
Epoch 10/20 - Train loss: 0.3259, Train Acc: 0.9014, Val Acc: 0.8555
Epoch 11/20 - Train loss: 0.3932, Train Acc: 0.8580, Val Acc: 0.8324
Epoch 12/20 - Train loss: 0.3410, Train Acc: 0.8841, Val Acc: 0.8497
Epoch 13/20 - Train loss: 0.3091, Train Acc: 0.8928, Val Acc: 0.8324
Epoch 14/20 - Train loss: 0.3888, Train Acc: 0.8594, Val Acc: 0.8092
Epoch 15/20 - Train loss: 0.2991, Train Acc

In [ ]:
# Save
torch.save(model.state_dict(), "model.pth")
torch.save({
    'epoch': EPOCHS,  # current epoch index (0-based)
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, 'checkpoint.pth')


In [ ]:
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for batch in val_loader:
        features, masks, labels = [x.to(device) for x in batch]
        logits = model(features, masks)
        preds = logits.argmax(1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)
test_acc = test_correct / test_total
print(f"Test accuracy: {test_acc*100:.2f}%")

#72.83 in in 10 epochs

Test accuracy: 90.17%


In [ ]:
import sys
sys.path.append(r"C:\Users\aliza\Desktop\hakathon\LSTMSequenceModel_.py")   



In [ ]:
from LSTMSequenceModel_ import LSTMSequenceModel

# Create model instance with the correct args
model = LSTMSequenceModel(num_features=NUM_FEATURES, max_seq_length=MAX_SEQ_LENGTH, num_classes=NUM_CLASSES)
# Load trained weights
model.load_state_dict(torch.load("model.pth", map_location=device))

# Send to GPU/CPU and set to eval mode
model.to(device).eval()


NameError: name 'NUM_CLASSES' is not defined

In [ ]:
import torch
import numpy as np
import os
import frame_constructor as fc  # Your feature extraction module

# Define constants (set these as per your setup)
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 10
MAX_SEQ_LENGTH = 300 # Maximum number of frames to use per video
NUM_FEATURES = 1280 # Number of features extracted by MobileNetV2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hardcoded class vocabulary corresponding to your label encoding
class_vocab = ["real", "fake"]  # index 0 = not_aggressive, 1 = aggressive
from LSTMSequenceModel_ import LSTMSequenceModel

# Create model instance with the correct args
model = LSTMSequenceModel(
    num_features=NUM_FEATURES,
    max_seq_length=MAX_SEQ_LENGTH,
    num_classes=len(class_vocab)
)
# Load trained weights
model.load_state_dict(torch.load("model.pth", map_location=device))

# Send to GPU/CPU and set to eval mode
model.to(device).eval()

# Your loaded PyTorch models
# feature_extractor: feature extraction model
# model: your LSTMSequenceModel or equivalent for classification
# Make sure they are both on the correct device
#model = "model.pth"  # Path to your trained model
feature_extractor.to(device).eval()
#model.to(device).eval()

def prepare_single_video(video_path):
    frames = fc.frame_cons(video_path)  # (num_frames, H, W, C)
    frame_mask = torch.zeros((1, MAX_SEQ_LENGTH), dtype=torch.bool)
    frame_features = torch.zeros((1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype=torch.float32)
    length = min(MAX_SEQ_LENGTH, frames.shape[0])
    for j in range(length):
        frame = frames[j]
        frame_tensor = torch.from_numpy(frame).permute(2, 0, 1).unsqueeze(0).float() / 255.0
        mean = torch.tensor([0.485, 0.456, 0.406], device=frame_tensor.device).view(1, 3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225], device=frame_tensor.device).view(1, 3, 1, 1)
        frame_tensor = (frame_tensor - mean) / std
        frame_tensor = frame_tensor.to(device)
        with torch.no_grad():
            feat = feature_extractor(frame_tensor)  # output shape (1, 2048)
        frame_features[0, j, :] = feat.squeeze(0).cpu()
    frame_mask[0, :length] = 1
    return frame_features, frame_mask


def sequence_prediction(video_path):
    frame_features, frame_mask = prepare_single_video(video_path)
    frame_features = frame_features.to(device)
    frame_mask = frame_mask.to(device)

    with torch.no_grad():
        logits = model(frame_features, frame_mask)
        probabilities = torch.softmax(logits, dim=-1).cpu().numpy()[0]

    print(f"Prediction for video: {video_path}")
    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i]*100:5.2f}%")

    return probabilities

# Example usage:
#video_path = r"C:\Users\aliza\Desktop\hakathon\archive\DFD_manipulated_sequences\26_12__outside_talking_still_laughing__LI63E4TS.mp4"
video_path=r'C:\Users\aliza\Desktop\hakathon\Alia Bhatt-fake.mp4'
sequence_prediction(video_path)


Prediction for video: C:\Users\aliza\Desktop\hakathon\Alia Bhatt-fake.mp4
  real: 91.86%
  fake:  8.14%


array([0.91859704, 0.08140292], dtype=float32)